#Classifying Flu Vaccine and H1N1 Vaccine (K-NN imputation)
##Final Project in CISC 451/839

In [2]:
#Importing necessary libraries

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
#from sklearnex import patch_sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report


#Reading in data
#data = pd.read_csv('/content/h1n1_flu.csv')
data = pd.read_csv('/content/drive/MyDrive/CISC451_DATASETS/h1n1_flu.csv')

In [3]:
#Declaring targets
targets = ['VACC_H1N1_F', 'VACC_SEAS_F']

In [4]:
#Dropping records where both targets have NaN
data = data.dropna(axis = 0, subset = targets, how = 'any')
print(data.shape)

In [5]:
#Dropping columns based on coverage
missing_val_threshold = 0.75  # 75% of data must be present in the feature for us to keep it 
missing_series = data.isnull().sum()/data.shape[0]  # count the number of nans in proportion to the total number of records
record_missing = pd.DataFrame(missing_series[missing_series > missing_val_threshold]).reset_index().rename(columns = {'index':'feature',0:'missing_val_fraction'})  # identify columns with more NaNs than the threshold
missing_val_drop = list(record_missing['feature']) 
df_cleaned_1 = data.copy()
df_cleaned_1 = df_cleaned_1.loc[:, ~df_cleaned_1.columns.isin(missing_val_drop)]  # delete the columns
print(len(missing_val_drop))
print(missing_val_drop)
print(df_cleaned_1.shape)

#Converting incorrect floats to ints


In [ ]:
#Dropping this column because it is unary
df_cleaned_1.drop(['MSA_DEF'], axis = 1, inplace=True) #DROP COMPLETELY, because unary

In [ ]:
#Casting all floats as int except "FLUWT", which needs to be a float
float64list = list(df_cleaned_1.loc[:, df_cleaned_1.dtypes == np.float64].columns) 
for x in float64list:
    df_cleaned_1[x] = np.floor(pd.to_numeric(df_cleaned_1[x], errors='coerce')).astype('Int64')
df_cleaned_1['FLUWT'] = df_cleaned_1['FLUWT'].astype('float64') 
df_cleaned_1.dtypes

Unnamed: 0      int64
INT_MONTH      object
LANGUAGE        int64
SAMP_DESIG     object
SEQNUMHH        int64
               ...   
CEN_REG         int64
FLUWT         float64
HHS_REGION      int64
MSA3_I          int64
STATE           int64
Length: 141, dtype: object

#Convert objects (label encoding)

In [ ]:
#Ssolating the object within the dataset
obj = ['SAMP_DESIG', 'SUBGROUP','INT_MONTH']
df_cleaned_1[obj] = df_cleaned_1[obj].astype('category')

In [ ]:
#Label encoding the converted object ones
df_cleaned_1['INT_MONTH'] = df_cleaned_1['INT_MONTH'].cat.codes
df_cleaned_1['SAMP_DESIG'] = df_cleaned_1['SAMP_DESIG'].cat.codes
df_cleaned_1['SUBGROUP'] = df_cleaned_1['SUBGROUP'].cat.codes

# KNN 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

df_cleaned_1["FLUWT"] = df_cleaned_1["FLUWT"].astype("float32") 

In [ ]:
np.isnan(df_cleaned_1).values.sum()

3018871

In [ ]:
df_cleaned_1.dtypes.value_counts()

Int64      122
int64       14
int8         3
float32      1
dtype: int64

In [ ]:
df_cleaned_1.shape

(70258, 140)

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
imputer = KNNImputer()
df_cleaned_1.shape
imputed_df = imputer.fit_transform(df_cleaned_1)

In [ ]:
imputed_df = pd.DataFrame(imputed_df)
imputed_df.columns = df_cleaned_1.columns
imputed_df.head()

INT_MONTH  LANGUAGE  SAMP_DESIG  SEQNUMHH   SEQNUMP  SUBGROUP  \
0        8.0       1.0         1.0   38548.0  385481.0       0.0   
1        8.0       1.0         1.0   18823.0  188231.0       0.0   
2        8.0       1.0         1.0   18823.0  188232.0       1.0   
3        7.0       1.0         1.0   54046.0  540461.0       0.0   
4        8.0       1.0         1.0   21986.0  219861.0       0.0   

   VACC1_SEAS_M  VACC1_SEAS_T  VACC1_SEAS_Y  VACC_H1N1_COUNT  ...  Q95_OCCPN  \
0          10.4           1.0        2009.0              0.0  ...        9.2   
1          10.0           1.0        2009.0              0.0  ...       13.6   
2          10.0           1.0        2009.0              0.0  ...       16.8   
3          10.0           1.0        2009.0              0.0  ...       13.0   
4           8.4           1.0        2009.2              0.0  ...       21.0   

   RACEETH4_I  RACE_I_R  RENT_OWN_R  SEX_I  CEN_REG    FLUWT  HHS_REGION  \
0         3.0       1.0         2.0    2.0      4.0    213.0         6.0   
1         4.0       3.0         2.0    2.0      2.0   5160.0         5.0   
2         1.0       3.0         2.0    2.0      2.0  20235.0         5.0   
3         3.0       1.0         1.0    2.0      4.0    804.0        10.0   
4         3.0       1.0         1.0    2.0      4.0   2849.0        10.0   

   MSA3_I  STATE  
0     1.0   35.0  
1     2.0   39.0  
2     2.0   39.0  
3     3.0   16.0  
4     1.0   41.0  

[5 rows x 140 columns]

In [ ]:
imputed_df.to_csv('imputed_data_h1n1_seas.csv')

In [ ]:
imputed_df.isnull().sum().value_counts()

0    140
dtype: int64